Let $n$ denote the number of teams in the tournament. Let $S_k$ be the total score of team $k$ where $k \in \{1, 2, 3, ... n\}$. Then, the balancing metric used to generate teams is defined as
$$\min_{i,j} \sum_{i=1}^{n}\sum_{j=i+1}^{n}|S_i - S_j|$$ 

In [108]:
from itertools import product, combinations
from operator import itemgetter

K = 20
data = ['2983', '2719', '2584', '2574', '2546', '2369', '2143', '2139',
'1835', '1750', '1697', '1823', '1602', '1600', '1397', '861']

restrictions = [
    ['2983','1697'],
    ['1823','2143'],
    ['1823','1835'],
    ['1835','2574'],
    ['1823','2574'],
    ['1823', '1600'],
    ['1602','2369'],
    ['1602','1600'],
    ['1397','1823']
]

prefer_nots = [
#     ['1835','2574'],
#     ['1623','1600'],
#     ['1623', '2574'],
#     ['1602','2369'],
#     ['1602','1600'],
#     ['1397','1623'],
#     ['2983','1697']
]

RECORDS = [
    ["LethalPilot", 2983],
    ["IvanTT", 2719],
    ["IceKing12323", 2584],
    ["SuperToad916", 2574],
    ["KirynMissy", 2546],
    ["humilau ", 2369],
    ["Star_Dragon4", 2143],
    ["Macrocosms", 2139],
    ["Labmonjo1210", 1835],
    ["Abach6", 1750],
    ["sapphronn", 1697],
    ["Dispuesto", 1823],
    ["AllieThaDon", 1602],
    ["josie_elite", 1600],
    ["Arua", 1397],
    ["cowdude12345", 861]
]

# Generates 1,000,000 possible teams

#Algorithm X functions
def solve(X, Y, solution):
    if X:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            yield from solve(X, Y, solution)
            deselect(X, Y, r, cols)
            solution.pop()
    else:
        yield list(solution)

def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

#Invert subset collection
def exact_cover(X, Y):
    newX = {j: set() for j in X}
    for i, row in Y.items():
        for j in row:
            newX[j].add(i)
    return newX

#----------------------------------------------------------------------

Y = {''.join(t): t for t in combinations(data, 4)}
X = exact_cover(data, Y)

all_teams = {}

for i, solution in enumerate(solve(X, Y, []), 1):
    add_tournament = True
    if i == 2_000_000:
        break
    current_team = [
        [solution[0][:4], solution[0][4:8], solution[0][8:12], solution[0][12:]], 
        [solution[1][:4], solution[1][4:8], solution[1][8:12], solution[1][12:]],
        [solution[2][:4], solution[2][4:8], solution[2][8:12], solution[2][12:]], 
        [solution[3][:4], solution[3][4:8], solution[3][8:12], solution[3][12:]]
    ]
    
    for team in current_team:
        for restriction in restrictions:
            if restriction[0] in team and restriction[1] in team:
                add_tournament = False
                
    if add_tournament:    
        all_teams[i] = current_team
    
# Computing the total differences for each possible team
team_number = 0
metric_dict = {}
for key, tournament in all_teams.items():
    total_difference = 0
    team_number += 1
    for k in range(4):
        left_array = [int(n) for n in tournament[k]] 
        for j in range(k+1, 4):
            right_array = [int(n) for n in tournament[j]]
            total_difference = total_difference + abs(sum(left_array) - sum(right_array))
    
    penalty = 0
    for team in tournament:
        for preference in prefer_nots:
            if preference[0] in team and preference[1] in team:
                penalty += 100
    
    metric_dict[key] = total_difference + penalty
    #metric_dict[team_number] = total_difference + penalty

# Find the k smallest differences
res = dict(sorted(metric_dict.items(), key = itemgetter(1))[:K])
counter = 1

def find_player_from_score(score):
    for player in RECORDS:
        if player[1] == score:
            return player[0]

# convert raw scores back to the playere names
tournaments = []
for key, value in res.items():
    current_tournament = []
    for team in all_teams[key]:
        current_team = []
        for player in team:
            current_team.append(find_player_from_score(int(player)))
        current_tournament.append(current_team)
    tournaments.append(current_tournament)
    counter += 1
    
balance_metric_scores = list(res.values())  

In [109]:
def get_score_from_player(name):
    for player in RECORDS:
        if player[0] == name:
            return player[1]

In [110]:
print("----------------------------")
for i, tournament in enumerate(tournaments):
    print("Possible Tournament", i + 1)
    print("Balance Metric Score:", balance_metric_scores[i])
    
    for k, team in enumerate(tournament):
        print("---")
        print("Team", k + 1)
        team_total = 0
        for l in range(4):
            team_total += get_score_from_player(team[l])
            print(team[l])
        print("Team Total Score:", team_total)
    print("----------------------------") 

----------------------------
Possible Tournament 1
Balance Metric Score: 36
---
Team 1
LethalPilot
Abach6
Dispuesto
AllieThaDon
Team Total Score: 8158
---
Team 2
IvanTT
Star_Dragon4
sapphronn
josie_elite
Team Total Score: 8159
---
Team 3
KirynMissy
humilau 
Labmonjo1210
Arua
Team Total Score: 8147
---
Team 4
IceKing12323
SuperToad916
Macrocosms
cowdude12345
Team Total Score: 8158
----------------------------
Possible Tournament 2
Balance Metric Score: 48
---
Team 1
LethalPilot
Abach6
Dispuesto
AllieThaDon
Team Total Score: 8158
---
Team 2
IceKing12323
SuperToad916
Star_Dragon4
cowdude12345
Team Total Score: 8162
---
Team 3
IvanTT
Macrocosms
sapphronn
josie_elite
Team Total Score: 8155
---
Team 4
KirynMissy
humilau 
Labmonjo1210
Arua
Team Total Score: 8147
----------------------------
Possible Tournament 3
Balance Metric Score: 66
---
Team 1
LethalPilot
IvanTT
josie_elite
cowdude12345
Team Total Score: 8163
---
Team 2
SuperToad916
Star_Dragon4
Abach6
sapphronn
Team Total Score: 8164
---